## MI-PAA, Úloha 2: Řešení problému batohu dynamickým programováním, metodou větví a hranic a aproximativním algoritmem

**Marián Hlaváč**, 11 Nov 2017 (hlavam30)  
marian.hlavac@fit.cvut.cz  
https://github.com/mmajko/knapsack-problem

## Zadání úlohy

- Naprogramujte řešení problému batohu:
  - metodou větví a hranic (B&B) tak, aby omezujícím faktorem byla hodnota optimalizačního kritéria. Tj. použijte ořezávání shora (překročení kapacity batohu) i zdola (stávající řešení nemůže být lepší než nejlepší dosud nalezené),
  - metodou dynamického programování (dekompozice podle kapacity nebo podle cen),
  - FPTAS algoritmem, tj. s použitím modifikovaného dynamického programování s dekompozicí podle ceny (při použití dekompozice podle kapacity není algoritmus FPTAS).



## Možné varianty řešení

Problém batohu je možné řešit hrubou silou, heuristicky, dynamickým programováním, algoritmem "meet-in-the-middle" a dalšími způsoby. 

V minulé úloze jsme se zabývali řešením hrubou silou a urychlení výpočtu pomocí heuristik. V této úloze bylo využito dvou způsobů urychlení výpočtu. Jedním ze způsobů je lepší prořezávání stavového prostoru. Druhým způsobem je využití dynamického programování a pomocí aproximačního schématu plně polynomiálního času (FPTAS - fully polynomial-time approx. scheme).

## Popis postupu řešení

Algoritmus a celý program poskytující výsledky je napsán v jazyce *Rust*. Program načte instance z datových souborů určených pro tuto úlohu a vypočte řešení všemi implementovanými metodami. Zapíše délku provádění výpočtu a všechna data poskytne v CSV formátu. Druhým nástrojem je *Jupyter Notebook*, který poskytnutá data vizualizuje.

Oproti minulé úlohy byl algoritmus hrubou silou přepsán z implementace pracující s bitovými maskami do rekurzivní varianty. Umožnilo to tak jednodušší implementaci Branch & Bound prořezávání.

Prořezávání stavového prostoru obohacené o Branch & Bound zajistí, že je prostor prořezán shora i zdola. Větve rekurze, které by přesáhly kapacitu batohu, nebo by neposkytly lepší výsledek, než dosavadní dosažený jsou ořezány.

Dynamické programování umožňuje zrychlení výpočtu na úkor pamětové náročnosti. Byla zvolena varianta dekompozice podle ceny, díky čemu pak lze snadno implementovat FPTAS.

Konečně, FPTAS představuje způsob, jak ovlivňovat "kvalitu" a rychlost výpočtu. Vstupní proměnnou je možné definovat maximální relativní chybu. Čím větší je povolená tato relativní chyba, tím by měl být výpočet teoreticky rychlejší.

### Kostra algoritmu

Celý algoritmus je k nahlédnutí ve [zdrojových souborech programu na GitHubu](https://github.com/mmajko/knapsack-problem). Pro rychlou představu je níže uveden úryvek z funkce mající na starost FPTAS výpočet.

```rust
...
    // Find the largest price in knapsack
    let max_price = knap.items.iter().fold(0, |acc, &x| if x.price > acc { x.price } else { acc });
    let ratio = (1.0 - accuracy) * max_price as f32 / knap.items.len() as f32;
    
    // Modify prices in knapsack
    let mut mod_knap = knap.clone();
    mod_knap.items = knap.items.iter().map(|item| { KnapItem { 
        id: item.id, weight: item.weight, price: f32::floor(item.price as f32 / ratio) as u16
    } }).collect();
    
    // Solve with dynamic solver
    let mut solution = solver_dynamic::solve(mod_knap);
...
```

Branch & Bound drží v paměti nejlepší dosažený výsledek a před každým sestoupením do větve rekurze zkontroluje cenu zbývajících předmětů, které nebyly do batohu přidány. Pokud je tato cena sečtená s cenou batohu nižší, než nejlepší dosažený výsledek, je jasné, že prohledáváním této větve se k lepšímu výsledku již nemůžeme dostat a můžeme rekurzi na tomto místě ukončit.

Dynamické programování přenáší výpočetní náročnosti na paměťovou náročnost. Držíme v paměti tabulku, do které ukládáme provedené výpočty. Konkrétně v této implementaci jde o dekompozici podle ceny. Do tabulky tak ukládáme výsledné váhy, sloupce představují ceny a řádky jednotlivé předměty. Výsledek pak nalezneme na posledním řádku v nejpravějším nenulovém sloupci.

FPTAS pak pomocí formule upravující ceny předmětů umožní vložit do výpočtu další proměnnou, kterou lze výpočet ovlivnit. Efektivně jde o zanedbávání určitého počtu bitů hodnot ceny. Výpočet je až na pozměněné ceny totožný s výpočtem pomocí dynamického programování s kompozicí podle ceny a je tak použita stejný kód implementace.

## Surová naměřená (raw) data

Níže uvedená tabulka je náhled na kompletní surová výstupní data z programu. Data lze získat jednoduchým způsobem (např. pro kontrolu) -- spuštěním skriptu `generate.sh`, který vytvoří soubor `results.csv` obsahující tato data.

### Sloupce

Názvy sloupců se vyskytují i dále v textu, zde je jejich stručný popis:

- **knap_id** - identifikátor instance
- **item_count** - počet předmětů (konfigurace instance)
- **method** - metoda výpočtu
  - *Recursive* - rekurzivní
  - *BranchAndBound* - prořezávání Branch & Bound
  - *Dynamic* - dynamickým programováním, dekompozice podle ceny
  - *FPTAS25* - FPTAS s 25% přesností (max. rel. chybou 0.75)
  - *FPTAS50* - FPTAS s 50% přesností (max. rel. chybou 0.50)
  - *FPTAS75* - FPTAS s 75% přesností (max. rel. chybou 0.25)
- **price** - vypočtená celková cena batohu
- **elapsed_ms** - doba výpočtu v milisekundách
- **optimal_price** - optimální cena batohu

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

# Konfigurace vizualizace dat
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (12.0, 6.0)
matplotlib.style.use('ggplot')
pd.options.display.max_rows = 15

# Čtení dat z běhu programu
raw = pd.read_csv("results.csv")
solutions = pd.read_csv("solutions.csv")

data = pd.merge(left=raw, right=solutions, on='knap_id').drop('item_count_y', axis=1)
data = data.rename(columns={
                'price_x': 'price', 
                'price_y': 'optimal_price', 
                'item_count_x': 'item_count'
            })

data

## Výsledky měření

Níže jsou uvedeny výsledky derivované z dat. Rychlosti řešení jsou seskupeny podle počtu předmětů a zprůměrovány. Výsledné průměrné časy jsou ke každé metodě řešení uvedeny jak tabulkou, tak grafem.

Měřítko grafů je lineární na vertikální ose.

### Průměrné rychlosti řešení pro všechny metody

V tabulce níže lze porovnat, jaké byly průměrné rychlosti výpočtu pro různé počty předmětů v batohu. Uvedené časy jsou v milisekundách.

In [ ]:
# Výpočet všech průměrů
methods = ['Recursive', 'BranchAndBound', 'Dynamic', 'FPTAS25', 'FPTAS50', 'FPTAS75']
methods_data = list(map(lambda x: data[data['method'] == x].groupby('item_count').mean()[['elapsed_ms']], methods))

# Sjednocení do jedné tabulky pro přehledný výpis
comp_time = pd.concat(methods_data, axis=1, join='inner')
comp_time.columns = ['recursive', 'branch&bound', 'dynamic', 'fptas25', 'fptas50', 'fptas75']

comp_time

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2)

axes[0, 0].set_title('Závislost času na poč. prvků: Rekurzivní výpočet')
axes[0, 0].plot(comp_time[['recursive']], marker = 'o', ls = '', c = (230/256, 64/256, 57/256))
axes[0, 1].set_title('Závislost času na poč. prvků: Branch & Bound')
axes[0, 1].plot(comp_time[['branch&bound']], marker = 'o', ls = '', c = (34/256, 139/256, 186/256))
axes[1, 0].set_title('Závislost času na poč. prvků: Dyn. programováním')
axes[1, 0].plot(comp_time[['dynamic']], marker = 'o', ls = '', c = (149/256, 143/256, 210/256))
axes[1, 1].set_title('Závislost času na poč. prvků: FPTAS (s 25% přesn.)')
axes[1, 1].plot(comp_time[['fptas25']], marker = 'o', ls = '', c = (145/256, 145/256, 145/256))


In [ ]:
# Vykreslení grafu znázorňující časovou náročnost pro všechny metody
comp_time.plot(
    title = 'Porovnání výpočetního času (v ms) všech metod na n (počtu předmětů)',
    kind = 'bar',
)

In [ ]:
# Vykreslení grafu znázorňující časovou náročnost pro všechny metody kromě hrubé síly
comp_time.loc[:, comp_time.columns != 'recursive'].plot(
    title = 'Porovnání výpočetního času (v ms) metod bez hrubé síly na n (počtu předmětů)',
    kind = 'bar',
)

Výsledky náročnosti výpočtu hrubou silou nejsou překvapivé, díky znalostem z minulé úlohy.

Pokud prozkoumáme pouze metody, které jsou oproti minulé úloze "nové", lze vypozorovat a potvrdit tak, že rychlost vzrůstu náročnosti výpočtu při vzrůstajícím n je stejná jako rychlost vzrůstu při hrubé síle, ovšem lze vypozorovat znatelnou redukci výpočetního času. Výpočet je až **25 x** rychlejší, než výpočet hrubou silou.

Ukazuje se, že metoda Branch & Bound může být často rychlejší, než řešení dynamickým programováním. Tento fakt nejvíce ovlivňuje charakteristika vstupních dat (optimální řešení se najde velmi brzo).

Z grafů lze vyčíst, že se mou implementací nepodařilo potvrdit, že by výpočetní náročnost klesala s větší povolenou relativní chybou.



### Porovnání relativní chyby u FPTAS

Na grafu níže lze vidět jaké byly relativní chyby pro jednotlivé přesnosti (max. relativní chyby) algoritmu FPTAS.

Sloupeček expected představuje očekávanou maximální relativní chybu. Maximum a minimum jsou krajní případy a mean je průměrná relativní chyba.

In [ ]:
pd.options.mode.chained_assignment = None

# Výpočet absolutních a relativních chyb u každého výpočtu
data_werr = data
data_werr['error'] = data_werr['optimal_price'] - data_werr['price']
data_werr['relative_error_%'] = (data_werr['error'] / data_werr['optimal_price']) * 100

expected = { 'FPTAS25': 75, 'FPTAS50': 50, 'FPTAS75': 25 }
minimum = { 'FPTAS25': 1, 'FPTAS50': 1, 'FPTAS75': 1 }
maximum = { 'FPTAS25': 0, 'FPTAS50': 0, 'FPTAS75': 0 }
means = { 'FPTAS25': 0, 'FPTAS50': 0, 'FPTAS75': 0 }
samples = { 'FPTAS25': 0, 'FPTAS50': 0, 'FPTAS75': 0 }

for index, row in data_werr.iterrows():
    if row['method'].startswith('FPTAS'):
        means[row['method']] += row['relative_error_%']
        samples[row['method']] += 1
        
        if row['relative_error_%'] < minimum[row['method']]:
            minimum[row['method']] = row['relative_error_%']
        
        if row['relative_error_%'] > maximum[row['method']]:
            maximum[row['method']] = row['relative_error_%']
            
for index, row in means.items():
    means[index] = row / samples[index]

relative_errors = pd.DataFrame({
    'minimum_rel_err_%': minimum,
    'mean_rel_err_%': means,
    'expected_rel_err_%': expected,
    'maximum_rel_err_%': maximum
})

# Výpis průměru a maxima relativní chyby pro každý počet předmětů
relative_errors

In [ ]:
relative_errors.plot(kind = 'bar')


Ukázalo se, že se průměrná chyba pohybovala daleko za hranicí očekávané relativní chyby, dokonce i maximální chyby se pohybovaly daleko od očekávané hranice.

Minimální chyba byla vždy nulová. Některé instance se podařilo spočítat k optimální hodnotě.

## Závěr

Urychlení výpočtu pomocí prořezávání a dynamickým programováním bylo úspěšné. Zrychlení výpočtu je daleko lepší, než zrychlení, kterého se podařilo dosáhnout v minulé úloze.

U FPTAS se ovšem už takového úspěchu pravděpodobně nedosáhlo. Zrychlení oproti ostatním metodám není znatelné, naopak se v některých případech ukázala časová náročnost jako vyšší.

Některé výpočty byly vypočteny skutečně s chybami, tyto chyby se průměrně pohybovaly kolem 2,6% (relativní chyba). Dalo by se však očekávat, že chyby budou daleko větší a více úzce kopírovat stanovenou hranici maximální relativní chyby. Zároveň se očekávalo další výrazné zrychlení. 

Vzhledem k tomu, že jsme byli schopní nalézt optimum v podobných časech, se FPTAS v tomto případě nevyplácí.

Zdrojové soubory úlohy lze najít na GitHubu. Link je uveden v hlavičce zprávy.